# lstm 모델 최종 결과

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import KFold

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def lstm_kfold_analysis(data_path, target_column, feature_columns, time_step=10, epochs=50, batch_size=64, n_splits=5, save_path=None):
    # Load and preprocess data
    data = pd.read_csv(data_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # Feature selection and scaling
    features = feature_columns
    target = target_column

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features + [target]])

    # Create KFold cross-validator
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    all_train_metrics = []
    all_test_metrics = []

    for train_index, test_index in kf.split(scaled_data):
        train_data, test_data = scaled_data[train_index], scaled_data[test_index]

        def create_dataset(data, time_step=1):
            X, Y = [], []
            for i in range(len(data) - time_step - 1):
                a = data[i:(i + time_step), :-1]
                X.append(a)
                Y.append(data[i + time_step, -1])
            return np.array(X), np.array(Y)

        X_train, y_train = create_dataset(train_data, time_step)
        X_test, y_test = create_dataset(test_data, time_step)

        # Reshape input to be [samples, time steps, features]
        X_train = X_train.reshape(X_train.shape[0], time_step, X_train.shape[2])
        X_test = X_test.reshape(X_test.shape[0], time_step, X_test.shape[2])

        # Build and train LSTM model
        model = build_lstm_model((time_step, X_train.shape[2]))
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

        # Predict and evaluate
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform predictions
        train_predict = scaler.inverse_transform(np.concatenate((np.zeros((train_predict.shape[0], len(features))), train_predict), axis=1))[:, -1]
        test_predict = scaler.inverse_transform(np.concatenate((np.zeros((test_predict.shape[0], len(features))), test_predict), axis=1))[:, -1]
        y_train_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_train.shape[0], len(features))), y_train.reshape(-1, 1)), axis=1))[:, -1]
        y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], len(features))), y_test.reshape(-1, 1)), axis=1))[:, -1]

        # Calculate metrics
        def calculate_metrics(true_values, predicted_values):
            mse = mean_squared_error(true_values, predicted_values)
            mae = mean_absolute_error(true_values, predicted_values)
            rmse = np.sqrt(mse)
            return mse, mae, rmse

        train_mse, train_mae, train_rmse = calculate_metrics(y_train_inv, train_predict)
        test_mse, test_mae, test_rmse = calculate_metrics(y_test_inv, test_predict)

        all_train_metrics.append((train_mse, train_mae, train_rmse))
        all_test_metrics.append((test_mse, test_mae, test_rmse))

        # Plotting
        plt.figure(figsize=(14, 5))
        plt.plot(data.index[:len(y_train_inv)], y_train_inv, color='blue', label='Train Actual')
        plt.plot(data.index[len(y_train_inv):len(y_train_inv) + len(y_test_inv)], y_test_inv, color='green', label='Test Actual')
        plt.plot(data.index[:len(train_predict)], train_predict, color='red', label='Train Predict')
        plt.plot(data.index[len(train_predict):len(train_predict) + len(test_predict)], test_predict, color='orange', label='Test Predict')
        plt.title(f'LSTM Prediction for {target_column} using {feature_columns}')
        plt.xlabel('Date')
        plt.ylabel(target_column)
        plt.legend()

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    avg_train_metrics = np.mean(all_train_metrics, axis=0)
    avg_test_metrics = np.mean(all_test_metrics, axis=0)

    print(f"Average Train MSE: {avg_train_metrics[0]}, MAE: {avg_train_metrics[1]}, RMSE: {avg_train_metrics[2]}")
    print(f"Average Test MSE: {avg_test_metrics[0]}, MAE: {avg_test_metrics[1]}, RMSE: {avg_test_metrics[2]}")

    return {
        'target': target_column,
        'avg_train_mse': avg_train_metrics[0],
        'avg_train_mae': avg_train_metrics[1],
        'avg_train_rmse': avg_train_metrics[2],
        'avg_test_mse': avg_test_metrics[0],
        'avg_test_mae': avg_test_metrics[1],
        'avg_test_rmse': avg_test_metrics[2]
    }

In [13]:
# Example usage:
variables = ['call_r', 'kospi', 'kosdaq', 'usdkrw', 'PPI', 'CPI', 'IPI', 'EPI',
             'TOTAL_foreign_own', 'KOSPI_foreign_own', 'KOSDAQ_foreign_own', 'CPI_ratio']
results = []

for var in variables:
    save_path = f'{var}_prediction.png'
    result = lstm_kfold_analysis('프로젝트데이터_외국인보유수_포함.csv', var, ['us_tb_3m', var], epochs=20, save_path=save_path)
    results.append(result)

# Displaying the results
for res in results:
    print(res)

Epoch 1/20


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0814
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0026
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0025
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0025
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0022
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0019
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0019
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0020
Epoch 9/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0017 
Epoch 10/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0020
Epoch 11/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0019
Epoch 12/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0017
Epoch 13/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0017
Epoch 14/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0019
Epoch 15/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0016
Epoch 16/20
73/73 ━━━━━━━━

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0580
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0031
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0025
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0024
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0021
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0020
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0023
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0020
Epoch 9/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0018
Epoch 10/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0020
Epoch 11/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0016
Epoch 12/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0017
Epoch 13/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0015
Epoch 14/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0016
Epoch 15/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0018
Epoch 16/20
73/73 ━━━━━━━

...

73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0694
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0103
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0089
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0077
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0064
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0065
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0062
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0059
Epoch 9/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0054
Epoch 10/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0059
Epoch 11/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0053
Epoch 12/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0061
Epoch 13/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0055
Epoch 14/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0053
Epoch 15/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0051
Epoch 16/20
73/73 ━━━━━━━━━━━━

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0461
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0105 
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0082
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0076
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0063
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0057
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0062
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0067
Epoch 9/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0059
Epoch 10/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0051
Epoch 11/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0056
Epoch 12/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0048
Epoch 13/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0054
Epoch 14/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0052
Epoch 15/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0054
Epoch 16/20
73/73 ━━━━━━━━━━

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0701
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0102
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0091
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0073
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0070
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0064
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0063
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0060
Epoch 9/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0056
Epoch 10/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0055
Epoch 11/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0062
Epoch 12/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0053
Epoch 13/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0059
Epoch 14/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0052
Epoch 15/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0051
Epoch 16/20
73/73 ━━━━━━━━━━━

In [14]:
for res in results:
    print(res)

{'target': 'call_r', 'avg_train_mse': 0.006278518931318028, 'avg_train_mae': 0.049690524150297975, 'avg_train_rmse': 0.0789362573251397, 'avg_test_mse': 0.020621095499630055, 'avg_test_mae': 0.08099361035360288, 'avg_test_rmse': 0.14356315297376585}
{'target': 'kospi', 'avg_train_mse': 1701.1164324776805, 'avg_train_mae': 30.76872692829027, 'avg_train_rmse': 41.08381117079586, 'avg_test_mse': 5837.366834450538, 'avg_test_mae': 55.44390205104662, 'avg_test_rmse': 76.3152613854884}
{'target': 'kosdaq', 'avg_train_mse': 344.23642319604954, 'avg_train_mae': 13.28184519275533, 'avg_train_rmse': 18.406115076090174, 'avg_test_mse': 1091.2087053960347, 'avg_test_mae': 23.439328533368375, 'avg_test_rmse': 32.99455070604065}
{'target': 'usdkrw', 'avg_train_mse': 162.94044222362214, 'avg_train_mae': 8.45307961002543, 'avg_train_rmse': 12.75026660204167, 'avg_test_mse': 560.2463916697193, 'avg_test_mae': 15.204374412872358, 'avg_test_rmse': 23.65881547128483}
{'target': 'PPI', 'avg_train_mse': 0.3

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import KFold

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(30, return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(0.3))
    model.add(LSTM(30, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def lstm_kfold_analysis(data_path, target_column, feature_columns, time_step=10, epochs=50, batch_size=64, n_splits=5, save_path=None):
    # Load and preprocess data
    data = pd.read_csv(data_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # Feature selection and scaling
    features = feature_columns
    target = target_column

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features + [target]])

    # Create KFold cross-validator
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    all_train_metrics = []
    all_test_metrics = []

    for train_index, test_index in kf.split(scaled_data):
        train_data, test_data = scaled_data[train_index], scaled_data[test_index]

        def create_dataset(data, time_step=1):
            X, Y = [], []
            for i in range(len(data) - time_step - 1):
                a = data[i:(i + time_step), :-1]
                X.append(a)
                Y.append(data[i + time_step, -1])
            return np.array(X), np.array(Y)

        X_train, y_train = create_dataset(train_data, time_step)
        X_test, y_test = create_dataset(test_data, time_step)

        # Reshape input to be [samples, time steps, features]
        X_train = X_train.reshape(X_train.shape[0], time_step, X_train.shape[2])
        X_test = X_test.reshape(X_test.shape[0], time_step, X_test.shape[2])

        # Build and train LSTM model
        model = build_lstm_model((time_step, X_train.shape[2]))
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

        # Predict and evaluate
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform predictions
        train_predict = scaler.inverse_transform(np.concatenate((np.zeros((train_predict.shape[0], len(features))), train_predict), axis=1))[:, -1]
        test_predict = scaler.inverse_transform(np.concatenate((np.zeros((test_predict.shape[0], len(features))), test_predict), axis=1))[:, -1]
        y_train_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_train.shape[0], len(features))), y_train.reshape(-1, 1)), axis=1))[:, -1]
        y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], len(features))), y_test.reshape(-1, 1)), axis=1))[:, -1]

        # Calculate metrics
        def calculate_metrics(true_values, predicted_values):
            mse = mean_squared_error(true_values, predicted_values)
            mae = mean_absolute_error(true_values, predicted_values)
            rmse = np.sqrt(mse)
            return mse, mae, rmse

        train_mse, train_mae, train_rmse = calculate_metrics(y_train_inv, train_predict)
        test_mse, test_mae, test_rmse = calculate_metrics(y_test_inv, test_predict)

        all_train_metrics.append((train_mse, train_mae, train_rmse))
        all_test_metrics.append((test_mse, test_mae, test_rmse))

        # Plotting
        plt.figure(figsize=(14, 5))
        plt.plot(data.index[:len(y_train_inv)], y_train_inv, color='blue', label='Train Actual')
        plt.plot(data.index[len(y_train_inv):len(y_train_inv) + len(y_test_inv)], y_test_inv, color='green', label='Test Actual')
        plt.plot(data.index[:len(train_predict)], train_predict, color='red', label='Train Predict')
        plt.plot(data.index[len(train_predict):len(train_predict) + len(test_predict)], test_predict, color='orange', label='Test Predict')
        plt.title(f'LSTM Prediction for {target_column} using {feature_columns}')
        plt.xlabel('Date')
        plt.ylabel(target_column)
        plt.legend()

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    avg_train_metrics = np.mean(all_train_metrics, axis=0)
    avg_test_metrics = np.mean(all_test_metrics, axis=0)

    print(f"Average Train MSE: {avg_train_metrics[0]}, MAE: {avg_train_metrics[1]}, RMSE: {avg_train_metrics[2]}")
    print(f"Average Test MSE: {avg_test_metrics[0]}, MAE: {avg_test_metrics[1]}, RMSE: {avg_test_metrics[2]}")

    return {
        'target': target_column,
        'avg_train_mse': avg_train_metrics[0],
        'avg_train_mae': avg_train_metrics[1],
        'avg_train_rmse': avg_train_metrics[2],
        'avg_test_mse': avg_test_metrics[0],
        'avg_test_mae': avg_test_metrics[1],
        'avg_test_rmse': avg_test_metrics[2]
    }

In [16]:
# Example usage:
variables = ['call_r', 'kospi', 'kosdaq', 'usdkrw', 'PPI', 'CPI', 'IPI', 'EPI',
             'TOTAL_foreign_own', 'KOSPI_foreign_own', 'KOSDAQ_foreign_own', 'CPI_ratio']
results = []

for var in variables:
    save_path = f'{var}_prediction.png'
    result = lstm_kfold_analysis('프로젝트데이터_외국인보유수_포함.csv', var, ['us_tb_3m', var], epochs=50, save_path=save_path)
    results.append(result)

Epoch 1/50


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0798
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0060
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0051
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0049
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0049
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0047
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0042
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0041
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0038
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0034
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0034
Epoch 13/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0035
Epoch 14/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0032
Epoch 15/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0032
Epoch 16/50
73/73 ━━━━━━━━━━━━

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0641
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0068
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0056
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0052
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0049
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0045
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0042
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0042
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0040
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0039
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0035
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0036
Epoch 13/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0033
Epoch 14/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0031
Epoch 15/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0031
Epoch 16/50
73/73 ━━━━━━━━━━━━

...

73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0743
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0146
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0119
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0104
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0101
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0089
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0089
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0080
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0083
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0085
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0077
Epoch 13/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0070
Epoch 14/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0068
Epoch 15/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0066
Epoch 16/50
73/73 ━━━━━━━━━━━━

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0825
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0163
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0133
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0110
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0090
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0097
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0091
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0091
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0080
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0084
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0080
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073
Epoch 13/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0074
Epoch 14/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0072
Epoch 15/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065
Epoch 16/50
73/73 ━━━━━━━━━━━━

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0977
Epoch 2/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0157
Epoch 3/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0132
Epoch 4/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0116
Epoch 5/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0096
Epoch 6/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0095
Epoch 7/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0081
Epoch 8/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0084
Epoch 9/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0089
Epoch 10/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073
Epoch 11/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0070
Epoch 12/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0073
Epoch 13/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0065
Epoch 14/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0072
Epoch 15/50
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0067
Epoch 16/50
73/73 ━━━━━━━━━━━━

In [17]:
# Displaying the results
for res in results:
    print(res)

{'target': 'call_r', 'avg_train_mse': 0.0063069139506366475, 'avg_train_mae': 0.050909522380387186, 'avg_train_rmse': 0.07912875860320373, 'avg_test_mse': 0.01948425446621229, 'avg_test_mae': 0.07922955935834901, 'avg_test_rmse': 0.13952697589034385}
{'target': 'kospi', 'avg_train_mse': 1911.8633067751161, 'avg_train_mae': 32.95313565045261, 'avg_train_rmse': 43.528314601700814, 'avg_test_mse': 5440.154794190296, 'avg_test_mae': 53.243374472917665, 'avg_test_rmse': 73.68803482479021}
{'target': 'kosdaq', 'avg_train_mse': 243.16760237962484, 'avg_train_mae': 11.14793268310584, 'avg_train_rmse': 15.57013788225484, 'avg_test_mse': 879.5938759936125, 'avg_test_mae': 20.872928321659465, 'avg_test_rmse': 29.656238014568792}
{'target': 'usdkrw', 'avg_train_mse': 159.17455788427137, 'avg_train_mae': 8.422210422488758, 'avg_train_rmse': 12.60115290677934, 'avg_test_mse': 508.9904917971523, 'avg_test_mae': 14.413418918096221, 'avg_test_rmse': 22.553890090659095}
{'target': 'PPI', 'avg_train_mse'

In [25]:
import pandas as pd

# 주어진 결과를 바탕으로 데이터프레임 생성
results = [
    {'target': 'call_r', 'avg_train_mse': 0.0063069139506366475, 'avg_train_mae': 0.050909522380387186, 'avg_train_rmse': 0.07912875860320373, 'avg_test_mse': 0.01948425446621229, 'avg_test_mae': 0.07922955935834901, 'avg_test_rmse': 0.13952697589034385},
    {'target': 'kospi', 'avg_train_mse': 1911.8633067751161, 'avg_train_mae': 32.95313565045261, 'avg_train_rmse': 43.528314601700814, 'avg_test_mse': 5440.154794190296, 'avg_test_mae': 53.243374472917665, 'avg_test_rmse': 73.68803482479021},
    {'target': 'kosdaq', 'avg_train_mse': 243.16760237962484, 'avg_train_mae': 11.14793268310584, 'avg_train_rmse': 15.57013788225484, 'avg_test_mse': 879.5938759936125, 'avg_test_mae': 20.872928321659465, 'avg_test_rmse': 29.656238014568792},
    {'target': 'usdkrw', 'avg_train_mse': 159.17455788427137, 'avg_train_mae': 8.422210422488758, 'avg_train_rmse': 12.60115290677934, 'avg_test_mse': 508.9904917971523, 'avg_test_mae': 14.413418918096221, 'avg_test_rmse': 22.553890090659095},
    {'target': 'PPI', 'avg_train_mse': 0.31813702410641653, 'avg_train_mae': 0.4470466369866436, 'avg_train_rmse': 0.5357610337913405, 'avg_test_mse': 0.6366271907604311, 'avg_test_mae': 0.6048776494497391, 'avg_test_rmse': 0.7796674713617194},
    {'target': 'CPI', 'avg_train_mse': 0.19292403566026645, 'avg_train_mae': 0.33959374853615837, 'avg_train_rmse': 0.43223267111663166, 'avg_test_mse': 0.3195022217489022, 'avg_test_mae': 0.42979657325608506, 'avg_test_rmse': 0.5528597632717211},
    {'target': 'IPI', 'avg_train_mse': 3.5902159474278497, 'avg_train_mae': 0.8491550839722116, 'avg_train_rmse': 1.8928388026671645, 'avg_test_mse': 12.081425834722172, 'avg_test_mae': 1.7590446464212026, 'avg_test_rmse': 3.4744685723698887},
    {'target': 'EPI', 'avg_train_mse': 2.147006424973475, 'avg_train_mae': 0.5756483582694468, 'avg_train_rmse': 1.4643393417515471, 'avg_test_mse': 7.738076429723989, 'avg_test_mae': 1.2246290777792292, 'avg_test_rmse': 2.781528522669641},
    {'target': 'TOTAL_foreign_own', 'avg_train_mse': 0.05463689075759524, 'avg_train_mae': 0.17104927637597206, 'avg_train_rmse': 0.23342261792977176, 'avg_test_mse': 0.1835489815904903, 'avg_test_mae': 0.3242444960116925, 'avg_test_rmse': 0.42836776514954344},
    {'target': 'KOSPI_foreign_own', 'avg_train_mse': 0.06208792130599635, 'avg_train_mae': 0.1866344873228259, 'avg_train_rmse': 0.24752912846691033, 'avg_test_mse': 0.19679389403872996, 'avg_test_mae': 0.33661619504639234, 'avg_test_rmse': 0.4433703976461187},
    {'target': 'KOSDAQ_foreign_own', 'avg_train_mse': 0.05988729269390146, 'avg_train_mae': 0.145665377490295, 'avg_train_rmse': 0.24282819244537085, 'avg_test_mse': 0.209310214010326, 'avg_test_mae': 0.26442740970990297, 'avg_test_rmse': 0.4573237711849365},
    {'target': 'CPI_ratio', 'avg_train_mse': 0.01307387966668567, 'avg_train_mae': 0.04642119327643505, 'avg_train_rmse': 0.11434039280604254, 'avg_test_mse': 0.05108110770028905, 'avg_test_mae': 0.11701408886556117, 'avg_test_rmse': 0.22600440676375605}
]

# 데이터프레임 생성
df = pd.DataFrame(results)

# CSV 파일로 저장
csv_path = 'lstm_kfold_results.xlsx'
df.to_excel(csv_path, index=False)


In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load and preprocess data
data_path = '프로젝트데이터_외국인보유수_포함.csv'
data = pd.read_csv(data_path)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Feature selection and scaling
def preprocess_data(data, feature_columns, target_column):
    features = feature_columns
    target = target_column

    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features + [target]])

    return scaled_data, scaler

# Example usage for preprocessing
variables = ['call_r', 'kospi', 'kosdaq', 'usdkrw', 'PPI', 'CPI', 'IPI', 'EPI',
             'TOTAL_foreign_own', 'KOSPI_foreign_own', 'KOSDAQ_foreign_own', 'CPI_ratio']
feature_columns = ['us_tb_3m']  # Example feature columns, can be adjusted
preprocessed_data = {}

for var in variables:
    scaled_data, scaler = preprocess_data(data, feature_columns, var)
    preprocessed_data[var] = (scaled_data, scaler)

In [19]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import KFold

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(30, return_sequences=True, input_shape=input_shape)))
    model.add(Dropout(0.3))
    model.add(LSTM(30, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), :-1]
        X.append(a)
        Y.append(data[i + time_step, -1])
    return np.array(X), np.array(Y)

def lstm_kfold_analysis(scaled_data, scaler, target_column, feature_columns, time_step=10, epochs=50, batch_size=64, n_splits=5, save_path=None):
    # Create KFold cross-validator
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    all_train_metrics = []
    all_test_metrics = []

    for train_index, test_index in kf.split(scaled_data):
        train_data, test_data = scaled_data[train_index], scaled_data[test_index]

        X_train, y_train = create_dataset(train_data, time_step)
        X_test, y_test = create_dataset(test_data, time_step)

        # Reshape input to be [samples, time steps, features]
        X_train = X_train.reshape(X_train.shape[0], time_step, X_train.shape[2])
        X_test = X_test.reshape(X_test.shape[0], time_step, X_test.shape[2])

        # Build and train LSTM model
        model = build_lstm_model((time_step, X_train.shape[2]))
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

        # Predict and evaluate
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # Inverse transform predictions
        train_predict = scaler.inverse_transform(np.concatenate((np.zeros((train_predict.shape[0], len(feature_columns))), train_predict), axis=1))[:, -1]
        test_predict = scaler.inverse_transform(np.concatenate((np.zeros((test_predict.shape[0], len(feature_columns))), test_predict), axis=1))[:, -1]
        y_train_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_train.shape[0], len(feature_columns))), y_train.reshape(-1, 1)), axis=1))[:, -1]
        y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((y_test.shape[0], len(feature_columns))), y_test.reshape(-1, 1)), axis=1))[:, -1]

        # Calculate metrics
        def calculate_metrics(true_values, predicted_values):
            mse = mean_squared_error(true_values, predicted_values)
            mae = mean_absolute_error(true_values, predicted_values)
            rmse = np.sqrt(mse)
            return mse, mae, rmse

        train_mse, train_mae, train_rmse = calculate_metrics(y_train_inv, train_predict)
        test_mse, test_mae, test_rmse = calculate_metrics(y_test_inv, test_predict)

        all_train_metrics.append((train_mse, train_mae, train_rmse))
        all_test_metrics.append((test_mse, test_mae, test_rmse))

        # Plotting
        plt.figure(figsize=(14, 5))
        plt.plot(data.index[:len(y_train_inv)], y_train_inv, color='blue', label='Train Actual')
        plt.plot(data.index[len(y_train_inv):len(y_train_inv) + len(y_test_inv)], y_test_inv, color='green', label='Test Actual')
        plt.plot(data.index[:len(train_predict)], train_predict, color='red', label='Train Predict')
        plt.plot(data.index[len(train_predict):len(train_predict) + len(test_predict)], test_predict, color='orange', label='Test Predict')
        plt.title(f'LSTM Prediction for {target_column} using {feature_columns}')
        plt.xlabel('Date')
        plt.ylabel(target_column)
        plt.legend()

        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()

    avg_train_metrics = np.mean(all_train_metrics, axis=0)
    avg_test_metrics = np.mean(all_test_metrics, axis=0)

    print(f"Average Train MSE: {avg_train_metrics[0]}, MAE: {avg_train_metrics[1]}, RMSE: {avg_train_metrics[2]}")
    print(f"Average Test MSE: {avg_test_metrics[0]}, MAE: {avg_test_metrics[1]}, RMSE: {avg_test_metrics[2]}")

    return {
        'target': target_column,
        'avg_train_mse': avg_train_metrics[0],
        'avg_train_mae': avg_train_metrics[1],
        'avg_train_rmse': avg_train_metrics[2],
        'avg_test_mse': avg_test_metrics[0],
        'avg_test_mae': avg_test_metrics[1],
        'avg_test_rmse': avg_test_metrics[2]
    }

In [ ]:
# Example usage:
results = []

for var in variables:
    scaled_data, scaler = preprocessed_data[var]
    save_path = f'{var}_prediction.png'
    result = lstm_kfold_analysis(scaled_data, scaler, var, feature_columns, epochs=50, save_path=save_path)
    results.append(result)

# Displaying the results
for res in results:
    print(res)